In [549]:
import dask.dataframe as dd
import dask.array as da
import dask
from dask.distributed import Client, TimeoutError, LocalCluster, progress
from dask.diagnostics import ProgressBar
import pandas as pd
import hvplot.dask
import hvplot.pandas
import re
from textblob import TextBlob
from langdetect import detect
from scipy.stats import f_oneway, ttest_ind

#code found on stack exchange
#makes a new client if one is not running, runs 4 workers
try:
    client = Client('tcp://localhost:8787', timeout='5s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=4, ip='localhost')
    client = Client(cluster)


client


<Client: 'tcp://127.0.0.1:8787' processes=4 threads=8, memory=16.00 GiB>

In [550]:
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

keyCats = {
    'gen' : ['mental health', 'trauma', 'psychosis', 'psychologist', 'psychotherapist', 'psychiatrist', 'psychotherapy'],
    'dep' : ['depression', 'depressed', 'depressant'],
    'anx' : ['anxiety', 'anxious'],
    'ocd' : ['OCD', 'obsessive-compulsive disorder'],
    'adhd' : ['ADHD', 'attention deficit'],
    'ptsd' : ['PTSD', 'posttraumatic stress disorder'],
    'bip' : ['bipolar'],
    'asd' : ['autism'],
    'schiz' : ['schizophrenia'],
    'ed' : ['eating disorder', 'anorexia', 'bulimia']
}

abbrCats = {
    'gen' : 'General',
    'dep' : 'Depression',
    'anx' : 'Anxiety',
    'ocd' : 'OCD',
    'adhd' : 'ADHD',
    'ptsd' : 'PTSD',
    'bip' : 'Bipolar',
    'asd' : 'Autism',
    'schiz' : 'Schizophrenia',
    'ed' : 'Eating Disorder'
}

In [551]:
#contains tweets (and time of the tweet) that contain mental health key word
#polled every 2 minutes
tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t')

#try reading in the preprocessed tweets
# if the csv does not exist, create it
try:
    # read the preprocessed csv
    tweetdf = dd.read_parquet(path='mental_health_tweets_preprocessed_en.parquet')
except:
    #contains tweets (and time of the tweet) that contain mental health key word
    #polled every 2 minutes
    tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t')
    
    #pre process
    # preprocess instructions found at https://youtu.be/ujId4ipkBio
    def preprocess(text):
        # remove mentions
        text = re.sub('@[a-zA-Z0-9]+', '', text)
        
        #  remove hashtag symbol
        text = re.sub('#', '', text)
        
        # remove retweet identifier
        text = re.sub('RT[\s]+', '', text)
        
        # remove links
        text = re.sub('https?:\/\/\S+', '', text)


        text = re.sub('[^a-zA-Z\\s]', '', text)

        text = text.strip()

        return text

    tweetdf['text'] = tweetdf['tweet'].map(preprocess)

    # remove any tweets that are either empty, or not in english
    tweetdf = tweetdf.loc[tweetdf['text'] != '']
    tweetdf['language'] = tweetdf['text'].map(lambda x: detect(x))
    tweetdf = tweetdf.loc[tweetdf['language'] == 'en']

    # drop original tweet and language columns
    tweetdf = tweetdf.drop(columns=['tweet', 'language'])

    #write pre processed text to parquet (dask recommends parquet)
    dd.to_parquet(tweetdf, path='mental_health_tweets_preprocessed_en.parquet')
    
    

In [552]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [553]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

# countdf = client.persist(countdf)

In [554]:
# countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)

In [555]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

# create a new column for each key, with values of whether or not
# the tweet in that row contains that key
for i in keyWords:
    tweetdf[i] = tweetdf['text'].map_partitions(containsKey, i)


In [556]:
#count tweets each keyword appears in
#if a tweet contains multiple keywords, then they are added 
#to the result df as |word1|word2| etc.

def keysInTweet(df):
    keys = ''
    for k in keyWords:
        if df[k]:
            keys = keys + '|' + k
    keys = keys + '|'
    return keys

tweetdf['keys'] = tweetdf.apply(keysInTweet, axis=1, meta=('keys', 'object'))

# drop tweets with no keys
tweetdf = tweetdf.loc[tweetdf['keys'] != '|']

# convert the value counts to its own df
multiKeyCount = tweetdf['keys'].value_counts().to_frame()
# copy the key column (which actually holds counts), rename it and then drop the original
multiKeyCount['count'] = multiKeyCount['keys']
multiKeyCount = multiKeyCount.drop(labels='keys', axis=1)

# multiKeyCount = client.persist(multiKeyCount)

In [557]:
# multiKeyCount.hvplot.bar(y='count', use_index=True)

In [558]:
# create an empty dask dataframe, with 1 parition (since max rows is 24)
keyCount = pd.DataFrame({'key' : keyWords})
keyCount = dd.from_pandas(keyCount, npartitions=1)

# given key k, find the number of times k appears in a tweet
def countKeys(k):
    return tweetdf[k].compute()

# create a new row of the key count
keyCount['count'] = keyCount['key'].map(countKeys, meta=('count', 'int64'))

# sort rows by descending count
keyCount = keyCount.sort_values(by='count', ascending=True)

# keyCount = client.persist(keyCount)

In [559]:
# keyCountPlot = keyCount.hvplot.bar(y='count', x='key').opts(fontscale=0.75, invert_axes=True)
# hvplot.save(keyCountPlot, 'keyCountPlot.png')

In [560]:
# determine whether each tweet has keywords related to each category
def getCategories(row, keyList):
    for k in keyList:
        if row[k]:
            return True
        
    return False


# create a new column for each category
for c in keyCats.keys():
    print(c)
    tweetdf[c] = tweetdf.apply(getCategories, args=([keyCats[c]]), axis=1, meta=(c, 'boolean'))

# drop the original keyword columns
tweetdf = tweetdf.drop(columns=keyWords)


gen
dep
anx
ocd
adhd
ptsd
bip
asd
schiz
ed


In [561]:
# get subjectivity of each tweet [0, 1]
def compSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# get polarity of tweet [-1, 1]
def compPolarity(text):
    return TextBlob(text).sentiment.polarity

# add subjectivity and polarity as new columns
tweetdf['subjectivity'] = tweetdf['text'].map(compSubjectivity)
tweetdf['polarity'] = tweetdf['text'].map(compPolarity)

# speeds up computation of mean and std of sub and pol for each cat
tweetdf.compute()
tweetdf = tweetdf.persist()

tweetdf

,time,text,year,month,day,hour,minute,keys,gen,dep,anx,ocd,adhd,ptsd,bip,asd,schiz,ed,subjectivity,polarity
npartitions=1,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],object,int64,int64,int64,int64,int64,object,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [562]:
# can store results in a pandas dataframe since we know it will fit in memory
mlres = pd.DataFrame()

# for each category, make a series of sub and pol mean and std
# then add it to the output pandas df
for c in keyCats.keys():
    tmp = tweetdf.loc[tweetdf[c]]
    data = (tmp['subjectivity'].mean().compute(),\
            tmp['polarity'].mean().compute(),\
            tmp['subjectivity'].std().compute(),\
            tmp['polarity'].std().compute()
            )
    ser = pd.Series(data=data, index=['subjectivity mean', 'polarity mean', 'subjectivity std', 'polarity std'], name=c)
    mlres = pd.concat([mlres, ser], axis=1)

mlres = mlres.rename(abbrCats)


mlres

,gen,dep,anx,ocd,adhd,ptsd,bip,asd,schiz,ed
subjectivity mean,0.395936,0.396557,0.429297,0.384950,0.456358,0.404048,0.432043,0.380166,0.391577,0.391295
polarity mean,0.022771,0.064290,0.112286,0.061012,0.068241,0.074570,0.021586,0.057209,0.023130,0.013195
subjectivity std,0.221135,0.281759,0.220136,0.268193,0.251877,0.269766,0.312199,0.298770,0.300826,0.314826
polarity std,0.219297,0.268468,0.229377,0.200037,0.253451,0.252619,0.300596,0.257552,0.257849,0.255851


In [563]:
# mlres.drop(index=['subjectivity std', 'polarity std', 'polarity mean']).hvplot.bar(use_index=True, title='Subjectivity Means', xlabel='Topic', ylabel='Subjectivity', ylim=(0, 1))

In [564]:
# mlres.drop(index=['subjectivity std', 'polarity std', 'subjectivity mean']).hvplot.bar(use_index=True, title='Polarity Means', xlabel='Topic', ylabel='Polarity', ylim=(-1, 1))

In [565]:
# One way ANOVA on column
def owANOVA(col):
    catArrs = []
    for c in keyCats.keys():
        tmp = tweetdf.loc[tweetdf[c]]
        tmp = tmp[col]
        tmp = tmp.to_dask_array()
        catArrs.append(tmp)

    return f_oneway(*catArrs)

print('One Way ANOVA Results:')
print(f"Subjectivity: {owANOVA('subjectivity')}")
print(f"Polarity: {owANOVA('polarity')}")

One Way ANOVA Results:
Subjectivity: F_onewayResult(statistic=9.350427528302216, pvalue=2.5029086691447977e-14)
Polarity: F_onewayResult(statistic=50.31081595000186, pvalue=2.6357967063028356e-91)


In [566]:
# test on each pair of categories
def tTest(col, a, b):
    a = tweetdf.loc[tweetdf[a]]
    a = a[col]
    a = a.to_dask_array()
    b = tweetdf.loc[tweetdf[b]]
    b = b[col]
    b = b.to_dask_array()
    return ttest_ind(a=a, b=b, equal_var=False)
    
def allTTests(col):
    out = []
    keys = list(keyCats.keys())
    for x in range(len(keys)):
        for y in range(x+1, len(keys)):
            out.append((keys[x], keys[y], tTest(col, keys[x], keys[y])))
    return out

subjT = allTTests('subjectivity')
polT = allTTests('polarity')

def statSigT(res, alpha):
    out = []
    for i in res:
        sig = i[2][1] < alpha
        isSig = (abbrCats[i[0]], abbrCats[i[1]], sig)
        out.append(isSig)
    return out

subjSig = statSigT(subjT, 0.05)
polSig = statSigT(polT, 0.05)

for i in range(len(subjSig)):
    print(f"Subjectivity: {subjSig[i]}")
    print(f"Polarity: {polSig[i]}")

Subjectivity Satistical Significance: 
Polarity Satistical Significance: 
Subjectivity: ('General', 'Depression', False)
Polarity: ('General', 'Depression', True)
Subjectivity: ('General', 'Anxiety', True)
Polarity: ('General', 'Anxiety', True)
Subjectivity: ('General', 'OCD', False)
Polarity: ('General', 'OCD', False)
Subjectivity: ('General', 'ADHD', True)
Polarity: ('General', 'ADHD', True)
Subjectivity: ('General', 'PTSD', False)
Polarity: ('General', 'PTSD', True)
Subjectivity: ('General', 'Bipolar', True)
Polarity: ('General', 'Bipolar', False)
Subjectivity: ('General', 'Autism', True)
Polarity: ('General', 'Autism', True)
Subjectivity: ('General', 'Schizophrenia', False)
Polarity: ('General', 'Schizophrenia', False)
Subjectivity: ('General', 'Eating Disorder', False)
Polarity: ('General', 'Eating Disorder', False)
Subjectivity: ('Depression', 'Anxiety', True)
Polarity: ('Depression', 'Anxiety', True)
Subjectivity: ('Depression', 'OCD', False)
Polarity: ('Depression', 'OCD', Fals